In [4]:
from ultralytics import YOLO
import os
import glob
from PIL import Image

# Load trained YOLOv8 model
model = YOLO("/media/toannn/A_STORAGE/github/Autotranslate_Manga/MangaAutoTranslator/mltranslator/models/inpainting/comic-speech-bubble-detector.pt").to('cuda')

In [8]:
# print(model.names)
def save_predictions_in_yolo_format(predictions, output_dir):
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    for prediction in predictions:
        file_path = os.path.join(output_dir, f"{prediction['filename']}.txt")
        with open(file_path, "w") as f:
            for box in prediction["boxes"]:
                f.write(f"{box['cls']} {box['x_center']} {box['y_center']} {box['width']} {box['height']}\n")

def get_prediction(output_dir):
    image_files = glob.glob(os.path.join(output_dir, "*.jpg"))
    predictions = []

    for image_file in image_files:
        image = Image.open(image_file)
        result = model.predict(image, conf=0.7, half=True, augment=False, verbose=False)[0]
        w, h = image.size
        boxes = []
        for box in result.boxes:
            x_center, y_center, width, height = box.xywh[0]
            cls = box.cls
            #Only keep text_bubble class
            if int(cls.cpu().numpy()) == 0:
                boxes.append({
                    "cls": int(cls.cpu().numpy()),
                    "x_center": x_center/w,
                    "y_center": y_center/h,
                    "width": width/w,
                    "height": height/h
                })

        predictions.append({
            "filename": os.path.splitext(os.path.basename(image_file))[0],
            "boxes": boxes
        })
    return predictions

In [11]:
output_dir = "/home/toannn/Desktop/data/Manhua_part_4/obj_train_data/"
predictions = get_prediction(output_dir)
save_predictions_in_yolo_format(predictions, output_dir)
print("Finished!")

/tmp/ipykernel_29461/3671622346.py:25: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  if int(cls.cpu().numpy()) == 0:
/tmp/ipykernel_29461/3671622346.py:27: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  "cls": int(cls.cpu().numpy()),


Finished!
